In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import itertools
import scipy.stats as stats
from scipy.special import expit
from scipy.optimize import minimize
from scipy.stats import wilcoxon, ttest_ind
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from scipy.optimize import minimize
from sklearn.utils import resample
from sklearn.ensemble._forest import _generate_unsampled_indices

In [53]:
class MCCRF:
    def __init__(self, n_trees=100, s=2, min_samples_leaf=1,combination=1, dacc='u65', data_name=None, random_state=42):
        # build a random forest using sklearn RandomForestClassifier
        self.n_trees = n_trees
        self.s = s
        self.combination = combination
        self.dacc = dacc
        self.data_name = data_name
        self.w = np.ones(n_trees)/n_trees
        self.classes = None
        self.n_class = None
        self.rf = RandomForestClassifier(n_estimators=n_trees, min_samples_leaf=min_samples_leaf, random_state=random_state, )
        
        
    def fit(self, train_x, train_y):
        self.rf.fit(train_x, train_y)
        
        self.classes = self.rf.classes_
        self.n_class = len(self.rf.classes_)

        n_sample_leaves = {}
        intervals_leaves = {}

        for t in range(self.n_trees):
            n_sample_leaves[t] = {}
            intervals_leaves[t] = {}
            tree = self.rf.estimators_[t]

            n_nodes = tree.tree_.node_count
            children_left = tree.tree_.children_left
            children_right = tree.tree_.children_right
            sample_count = tree.tree_.value.reshape((-1, self.n_class))

            for i in range(n_nodes):
                is_leaf_node = (children_left[i] == children_right[i])
                if is_leaf_node:
                    n_sample_leaves[t][i] = sample_count[i]
                    n_total_sample = sum(sample_count[i])
                    intervals = sample_count[i].repeat(2).reshape(self.n_class, 2)
                    intervals[:,0] = intervals[:,0]/(n_total_sample + self.s)
                    intervals[:,1] = (intervals[:,1] + self.s)/(n_total_sample + self.s)
                    intervals_leaves[t][i] = intervals
                    
        self.intervals_leaves = intervals_leaves

    
    def instance_interval_dominance(self, instance_intervals):
        n_class = len(instance_intervals)
        decision = []
        for i in range(n_class):
            other_classes = np.setdiff1d(np.arange(n_class), np.array([i]))
            if np.any(instance_intervals[i, 1] < instance_intervals[other_classes, 0]):
                continue
            else:
                decision.append(i)
                
        return decision
    
    
    def mva(self, intervals):
        # intervals here is numpy array of shape (T, n_class, 2)
        vote_against = np.zeros(self.n_class)
        for t in range(self.n_trees):
            t_non_dominated_class = self.instance_interval_dominance(intervals[t])
            t_dominated_class = np.setdiff1d(np.arange(self.n_class), np.array(t_non_dominated_class))
            for c in t_dominated_class:
                vote_against[c] += 1
        mva = vote_against.min()
        predictions_index = np.where(vote_against==mva)[0]
        
        return self.classes[predictions_index]
                
    
    def ave(self, intervals):
        # intervals here is numpy array of shape (T, n_class, 2)
        ave_intervals = intervals.mean(axis=0)
        predictions_index = self.instance_interval_dominance(ave_intervals)
        
        return self.classes[predictions_index]
    
    
    def mldu_vote(self, intervals, dacc):
        # intervals here is numpy array of shape (T, n_class, 2)
        mass_function = {}
        for t in range(self.n_trees):
            t_non_dominated_class = tuple(self.instance_interval_dominance(intervals[t]))
            if t_non_dominated_class not in list(mass_function.keys()):
                mass_function[t_non_dominated_class] = 0
            mass_function[t_non_dominated_class] += 1/self.n_trees

        
        max_ledu = 0
        prediction_index = None
        focal_elements = list(mass_function.keys())
#         print(mass_function)
        for l in range(1, self.n_class+1):
            if len(subset_of_omega)> 5:
                return self.classes[list(prediction_index)]
            for subset_of_omega in itertools.combinations(np.arange(self.n_class), l):
                if len(subset_of_omega) == 0:
                    continue
                
                bel = 0
                for focal_element in focal_elements:
                    if set(focal_element).issubset(subset_of_omega):
                        bel += mass_function[focal_element]
                        
                if dacc == 'u80':
                    ledu = bel * (-1.2/(l**2) + 2.2/l)
                else:
                    # defautl u65
                    ledu = bel * (-0.6/(l**2) + 1.6/l)

#                     print(subset_of_omage, bel, ledu)

                if ledu > max_ledu:
                    max_ledu = ledu
                    prediction_index = subset_of_omega

            if max_ledu > -0.6/((l+1)**2) + 1.6/(l+1) or max_ledu < 0.1:
                break
        return self.classes[list(prediction_index)]
    
    
    def mldu_ave(self, intervals, dacc):
        # intervals here is numpy array of shape (T, n_class, 2)
        ave_intervals = intervals.mean(axis=0)
        bels = ave_intervals[:,0]
        class_order = np.argsort(-bels)
        max_ldu = 0
        for l in range(1, self.n_class+1):
            if l == self.n_class:
                bel = 1
            else:
                bel = bels[class_order[:l]].sum()
            if dacc == 'u80':
                ldu = bel * (-1.2/(l**2) + 2.2/l)
            else:
                # defautl u65
                ldu = bel * (-0.6/(l**2) + 1.6/l)
            if ldu > max_ldu:
                max_ldu = ldu
                predictions_index = class_order[:l]
        return self.classes[predictions_index]
    
    
    
    def predict(self, X, dacc=None):
        if X.ndim == 1:
            X = X.reshape(1, -1)
        if dacc is None:
            dacc = self.dacc
        predictions = []
        n_instance = X.shape[0]
        leaves_index = self.rf.apply(X)
        
        # get all [bel, pl] intervals for all instances, shape of (n_instance, T, c_class, 2)
        all_intrvals = np.zeros((n_instance, self.n_trees, self.n_class, 2))
        for i in range(n_instance):
            for t in range(self.n_trees):
                all_intrvals[i, t] = self.intervals_leaves[t][leaves_index[i,t]]
                
        if self.combination == 1:
            # MVA
            predictions = []
            for i in range(n_instance):
                predictions.append(self.mva(all_intrvals[i]))
            return predictions
        
        if self.combination == 2:
            # AVE
            predictions = []
            for i in range(n_instance):
                predictions.append(self.ave(all_intrvals[i]))
            return predictions
            
        if self.combination == 3:
            # generalized vote
            predictions = []
            for i in range(n_instance):
                predictions.append(self.mldu_vote(all_intrvals[i], dacc))
            return predictions
        
        if self.combination == 4:
            # generalized ave
            predictions = []
            for i in range(n_instance):
                predictions.append(self.mldu_ave(all_intrvals[i], dacc))
            return predictions
    
        
        
    def evaluate(self, X_test, y_test):
        # get both imprecise and precise predictions 
        predictions = self.predict(X_test)
        determinacy = 0
        single_set_accuracy = 0
        set_accuracy = 0
        set_size = 0
        u65 = 0
        u80 = 0
        for i in range(len(y_test)):
            prediction = predictions[i]
            if len(prediction) == 1:
                determinacy += 1
                if prediction[0] == y_test[i]:
                    single_set_accuracy += 1
                    u65 += 1
                    u80 += 1
            else:
                set_size += len(prediction)
                if y_test[i] in prediction:
                    set_accuracy += 1
                    u65 += (-0.6/(len(prediction)**2) + 1.6/len(prediction))
                    u80 += (-1.2/(len(prediction)**2) + 2.2/len(prediction))
                    
        n_determinate = determinacy
        n_indeterminate = len(y_test) - determinacy
        
        determinacy /= len(y_test)
        single_set_accuracy /= n_determinate
        if n_indeterminate == 0:
            set_accuracy = -1
            set_size = -1
        else:
            set_accuracy /= n_indeterminate
            set_size /= n_indeterminate
        u65 /= len(y_test)
        u80 /= len(y_test)
                
        return {'determinacy': determinacy,
                 'single set accuracy': single_set_accuracy,
                 'set accuracy': set_accuracy,
                 'set size': set_size,
                 'u65 score': u65, 
                 'u80 score': u80}

In [65]:
data = pd.read_csv("data/vehicle.csv")
X = np.array(data.iloc[:,:-1])
y = np.array(data.iloc[:,-1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [66]:
crf = MCCRF(n_trees=100, s=1, min_samples_leaf=1,combination=1, data_name=None, random_state=42)
crf.fit(X_train, y_train)

In [67]:
crf.combination = 1
eva = crf.evaluate(X_test, y_test)
eva

{'determinacy': 0.9928571428571429,
 'single set accuracy': 0.7733812949640287,
 'set accuracy': 1.0,
 'set size': 2.0,
 'u65 score': 0.7725000000000001,
 'u80 score': 0.7735714285714287}

## Basic Data

In [ ]:
data_names = ['balance_scale', 'ecoli', 'optdigits', 'page_blocks',
             'pendigits', 'segment', 'vehicle', 'vowel', 'waveform', 'wine']
data_names = ['letter']
combinations = [1, 2, 3, 4]
it = 10
k = 10

for d in range(len(data_names)):
    data_name = data_names[d]
    data = pd.read_csv('data/{}.csv'.format(data_name))
    X = np.array(data.iloc[:,:-1])
    y = np.array(data.iloc[:,-1])
    evaluation_for_data = np.zeros((6, it*k, 4))
    for i in tqdm(range(it)):
        kf = KFold(n_splits=k, shuffle=True)
        j = 0
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            crf = MCCRF(n_trees=100, s=2, min_samples_leaf=1,combination=1, data_name=None, random_state=None)
            crf.fit(X_train, y_train)
            
            for c in combinations:
                crf.combination = c
                eva = crf.evaluate(X_test, y_test)
                eva = np.array(list(eva.values())).round(4)
                evaluation_for_data[:,i*k+j ,c-1] = eva
            
            j += 1
    
#     np.save('results/{}_evaluation.npy'.format(data_name), evaluation_for_data)
print(evaluation_for_data.mean(axis=1))

## Noisy Data

In [96]:
data_names = ['balance_scale', 'ecoli', 'optdigits', 'page_blocks',
             'pendigits', 'segment', 'vehicle', 'vowel', 'waveform', 'wine']
noise_levels = [5, 10, 15, 20]
combinations = [1,2,3,4]
it = 10
k = 10

for noise_level in noise_levels:
    print('Noise Level:', noise_level)
    for d in range(len(data_names)):
        data_name = data_names[d]
        print(d, data_name)
        data = pd.read_csv('data/{}.csv'.format(data_name))
        X = np.array(data.iloc[:,:-1])
        y = np.array(data.iloc[:,-1])
        classes = np.unique(y)
        evaluation_for_data = np.zeros((6, it*k, 4))
        for i in tqdm(range(it)):
            kf = KFold(n_splits=k, shuffle=True)
            j = 0
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # in X_train, choose certain proportion of instance to change its label
                instance_select = np.random.choice(len(y_train),int(noise_level*len(y_train)/100),replace=False)
                for instance_index in instance_select:
                    candidate_y = np.setdiff1d(classes, y_train[instance_index])
                    y_train[instance_index] = candidate_y[np.random.choice(len(candidate_y), 1)[0]]
                    
                crf = MCCRF(n_trees=100, s=2, min_samples_leaf=1,combination=1, data_name=None, random_state=None) #dacc='u65'
                crf.fit(X_train, y_train)

                for c in combinations:
                    crf.combination = c
                    eva = crf.evaluate(X_test, y_test)
                    eva = np.array(list(eva.values())).round(4)
                    evaluation_for_data[:,i*k+j ,c-1] = eva

                j += 1

        np.save('results/{}_noise/{}_evaluation.npy'.format(noise_level,data_name), evaluation_for_data)

Noise Level: 5
0 balance_scale


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:56<00:00, 17.67s/it]


1 ecoli


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:37<00:00, 21.78s/it]


2 optdigits


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [15:32<00:00, 93.27s/it]


8 waveform


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:45<00:00, 58.58s/it]


9 wine


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:57<00:00,  5.76s/it]
